In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2022-12-22 10:43:38.478197: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
from tensorflow import keras

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'

In [5]:
#splitting train_full in train(0.8) and val(0.2)
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

In [6]:
# ### DATA AUGMENTATION ###
input_size = 299


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    zoom_range=0.1,
    horizontal_flip=True,
    #vertical_flip=True,
    rotation_range=5.0,
    fill_mode='nearest',
    #width_shift_range=0.1,
    #height_shift_range=0.1
    #channel_shift_range=0.2
    shear_range=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(input_size, input_size),
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(input_size, input_size),
    batch_size=32,
)

Found 4447 validated image filenames belonging to 6 classes.
Found 1112 validated image filenames belonging to 6 classes.


In [7]:
base_model = Xception(
    weights='imagenet',
    input_shape=(input_size, input_size, 3),
    include_top=False
)

base_model.trainable = False

inputs = keras.Input(shape=(input_size, input_size, 3))
    
base = base_model(inputs, training=False)
vector = keras.layers.GlobalAveragePooling2D()(base)

inner = keras.layers.Dense(100, activation='relu')(vector)
drop = keras.layers.Dropout(rate=0.5)(inner)

outputs = keras.layers.Dense(6)(drop)

model = keras.Model(inputs, outputs)

2022-12-22 10:43:58.224249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-22 10:43:58.231318: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-22 10:43:58.231936: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-22 10:43:58.232872: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [8]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


model.fit(train_generator, epochs=5, verbose=1, validation_data=val_generator)

Epoch 1/5


2022-12-22 10:44:12.759756: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-12-22 10:44:13.431253: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-22 10:44:13.431796: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-22 10:44:13.431843: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-12-22 10:44:13.432455: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-22 10:44:13.432532: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


139/139 [==============================] - 149s 1s/step - loss: 0.3636 - accuracy: 0.8826 - val_loss: 0.1472 - val_accuracy: 0.9532
Epoch 2/5
139/139 [==============================] - 137s 985ms/step - loss: 0.1784 - accuracy: 0.9413 - val_loss: 0.1235 - val_accuracy: 0.9577
Epoch 3/5
139/139 [==============================] - 138s 989ms/step - loss: 0.1457 - accuracy: 0.9521 - val_loss: 0.1257 - val_accuracy: 0.9559
Epoch 4/5
139/139 [==============================] - 137s 988ms/step - loss: 0.1247 - accuracy: 0.9606 - val_loss: 0.1210 - val_accuracy: 0.9523
Epoch 5/5
139/139 [==============================] - 136s 977ms/step - loss: 0.1197 - accuracy: 0.9645 - val_loss: 0.1164 - val_accuracy: 0.9577


In [9]:
#unfreeze the last 32 layers of the base model for doing a partial fine tuning 
base_model.trainable = True
for layer in base_model.layers[:-32]:
  layer.trainable = False



model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # Low learning rate to avoid destruction of the learned weights
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "kitchenware_final_{epoch:02d}_{val_accuracy:.3f}.h5",
        monitor="val_accuracy",
        verbose=1,
        save_best_only=True,
        mode='max'
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        verbose=1, 
        patience=5
    )
]

history_6 = model.fit(train_generator, epochs=50, verbose=0, validation_data=val_generator, callbacks=callbacks)


Epoch 1: val_accuracy improved from -inf to 0.96043, saving model to kitchenware_final_01_0.960.h5

Epoch 2: val_accuracy did not improve from 0.96043

Epoch 3: val_accuracy improved from 0.96043 to 0.96133, saving model to kitchenware_final_03_0.961.h5

Epoch 4: val_accuracy improved from 0.96133 to 0.96313, saving model to kitchenware_final_04_0.963.h5

Epoch 5: val_accuracy improved from 0.96313 to 0.96673, saving model to kitchenware_final_05_0.967.h5

Epoch 6: val_accuracy did not improve from 0.96673

Epoch 7: val_accuracy did not improve from 0.96673

Epoch 8: val_accuracy did not improve from 0.96673

Epoch 9: val_accuracy improved from 0.96673 to 0.96853, saving model to kitchenware_final_09_0.969.h5

Epoch 10: val_accuracy improved from 0.96853 to 0.96942, saving model to kitchenware_final_10_0.969.h5
Epoch 10: early stopping


In [11]:
#predictions for a single image (useful for testing service deployment)

from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.xception import decode_predictions

img_path = 'data/images/3962.jpg'

img = image.load_img(img_path, target_size=(299, 299))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
#print('Keras Predicted:', decode_predictions(preds, top=3)[0])
print(preds)

1/1 [==============================] - 0s 20ms/step
[[-4.701105   1.4049903 -3.1885097 14.954982  -5.150577   2.9660013]]


In [14]:
df_test = pd.read_csv('data/test.csv', dtype={'Id': str})
df_test['filename'] = 'data/images/' + df_test['Id'] + '.jpg'

In [15]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

#class_mode = 'input'means that in the label arrays that are returned each label will be will be images identical to input images
#useful for fitting autoencoders

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    #target_size=(150, 150),
    target_size=(input_size, input_size),
    batch_size=32,
    shuffle=False
)

Found 3808 validated image filenames.


In [17]:
y_pred = model.predict(test_generator)

119/119 [==============================] - 58s 478ms/step


In [18]:
classes = np.array(list(train_generator.class_indices.keys()))

In [19]:
predictions = classes[y_pred.argmax(axis=1)]